In [2]:
import nibabel as nib
import numpy as np
import matplotlib.pyplot as plt
import os
import surfdist as sd
from surfdist import viz, load, utils, analysis
import hcp_utils as hcp
import pandas as pd
from scipy import stats
from nilearn.plotting import plot_surf_stat_map
from itertools import combinations


def get_parcel(label,path_to_dlabel,brain_structure):
    ### Get the vertex index of nodes included in a specified parcel
    # label: label of the parcel of which you need the nodes
    # path_to_dlabel: path to a cifti2.dlabel.nii file
    # brain_structure: cifti2 brain structure
    
    cifti = nib.load(path_to_dlabel)
    brain_model = [x for x in cifti.header.get_index_map(1).brain_models if x.brain_structure==brain_structure][0]
    offset = brain_model.index_offset
    count = brain_model.index_count
    vertices = np.array(brain_model.vertex_indices[0:])

    label_map = pd.Series(cifti.get_fdata().squeeze()[offset:offset+count])

    label_lst = pd.DataFrame(cifti.header.get_axis(0).get_element(0)[1].values(),columns=('lab','col')).reset_index()
    label_tmp = label_lst[label_lst['lab'].isin(pd.Series(label))]['index'].values

    del cifti
    return vertices[label_map.isin(label_tmp)]


def eq_dist(seeds,surface,labels,hemi):
    ### Calculate the level of "equidistance" from 2 or more parcels
    # seeds: list of labels to use as seeds (must match the provided atlas)
    # surface: path to surface file (surf.gii)
    # labels: path to labels (cifti2.dlabel.nii)
    # hemi: 'L' or 'R'
    
    surf = nib.load(surface).darrays
    coord = [surf[0].data,surf[1].data]
    nodes = np.unique(surf[1].data)
    structure = ('CIFTI_STRUCTURE_CORTEX_LEFT' if hemi=='L' else 'CIFTI_STRUCTURE_CORTEX_RIGHT')
    # calculate distance maps from each seed region
    dists = []
    for src_tmp in seeds:
        src_vtx = get_parcel(src_tmp,labels,structure)
        dist_tmp = stats.zscore(sd.analysis.dist_calc(coord, nodes, src_vtx))
        dists.append(dist_tmp)
    # calculate the 'equidistance' measure
    pairs = list(combinations(np.arange(len(seeds)),2))
    equidistance_map = np.zeros(len(nodes))
    for idx0, idx1 in pairs:
        eq_tmp = np.abs(dists[idx0]-dists[idx1])
        equidistance_map += eq_tmp
    del surf
    return equidistance_map


### Store gradients in cifti2 dscalar.nii
def mk_grad1_dscalar(subject_ID, grads, template_cifti, output_dir):
    # subject_ID: string array e.g. "100206"
    # grads: array with dimensions gradients X vertices
    # template_cifti: any cifti2 file with a BrainModelAxis
    # output_dir: path to output directory
    data = np.zeros([grads.shape[0],template_cifti.shape[1]])
    data[0:,0:grads.shape[1]] = grads

    map_labels = ['eqd' for i in range(grads.shape[0])]
    ax0 = nib.cifti2.cifti2_axes.ScalarAxis(map_labels)
    ax1 = nib.cifti2.cifti2_axes.from_index_mapping(template_cifti.header.get_index_map(1))
    nifti_hdr = template_cifti.nifti_header
    del template_cifti
    
    new_img = nib.Cifti2Image(data, header=[ax0, ax1],nifti_header=nifti_hdr)
    new_img.update_headers()

    new_img.to_filename(f"{output_dir}{subject_ID}_eqd.dscalar.nii")

/home/fralberti/.local/lib/python3.8/site-packages/nilearn/datasets/__init__.py:93: FutureWarning: Fetchers from the nilearn.datasets module will be updated in version 0.9 to return python strings instead of bytes and Pandas dataframes instead of Numpy arrays.
  warn("Fetchers from the nilearn.datasets module will be "
pixdim[1,2,3] should be non-zero; setting 0 dims to 1


In [3]:
### VARIABLES TO SET BEFORE RUNNING
# directory containing subdirectories named fter subject IDs that contain the timeseries and surface files
root_dir = "/home/fralberti/Data/HCP/"
# directory where all intermediate files and the final output will be saved
output_dir = "/home/fralberti/Data/Gradientile/"
# list of IDs of subjects to include in the analyses
subj_id = np.array(["100206","100307","100408","100610","101006",
                    "101107","101309","101410","101915","102008",
                    "102109","102311","102513","102614","102715"])

In [6]:

template_cifti = nib.load(f'{root_dir}HCP_S1200_GroupAvg_v1/S1200.All.midthickness_MSMAll_va.32k_fs_LR.dscalar.nii')
eq_all = []
for subj in subj_id:
    eq_dist_LR = np.zeros([1,template_cifti.shape[1]])
    hemisphere = ['left','right']
    cifti_structure = ['CIFTI_STRUCTURE_CORTEX_LEFT','CIFTI_STRUCTURE_CORTEX_RIGHT']
    
    for i, hemi in enumerate(['L','R']):
#         seeds = [f'{hemi}_S_calcarine',f'{hemi}_S_central',[f'{hemi}_S_temporal_transverse',f'{hemi}_G_temp_sup-G_T_transv']]
        seeds = [f'{hemi}_S_calcarine', f'{hemi}_S_central',f'{hemi}_G_temp_sup-G_T_transv']

        surface = f'/home/fralberti/Data/HCP_zone_prim/{subj}/{subj}.{hemi}.midthickness_MSMAll.32k_fs_LR.surf.gii'
        labels = f'/home/fralberti/Data/HCP_zone_prim/{subj}/{subj}.aparc.a2009s.32k_fs_LR.dlabel.nii'

        eq = eq_dist(seeds,surface,labels,hemi)
        
        bm = [x for x in template_cifti.header.get_index_map(1).brain_models if x.brain_structure==cifti_structure[i]][0]
        eq_dist_LR[0,bm.index_offset:bm.index_offset+bm.index_count] = eq[bm.vertex_indices]
#         if subj in ["100206","100307","100408"]:
#             cmap='BrBG'
#             fig, axs = plt.subplots(1, 2, figsize=(8,4), subplot_kw={'projection':'3d'}, sharey=True)
#             plot_surf_stat_map(f'{root_dir}{subj}/T1w/fsaverage_LR32k/{subj}.{hemi}.midthickness_MSMAll.32k_fs_LR.surf.gii',
#                                eq, hemi=hemisphere[i], axes=axs[0], colorbar=False, cmap=cmap)
#             plot_surf_stat_map(f'{root_dir}{subj}/T1w/fsaverage_LR32k/{subj}.{hemi}.midthickness_MSMAll.32k_fs_LR.surf.gii',
#                                eq, hemi=hemisphere[i], axes=axs[1], colorbar=True, cmap=cmap, view='medial')
#             fig.suptitle(f'subj {subj} {hemi}', fontsize=12)
    
    eq_all.append(eq_dist_LR)
    mk_grad1_dscalar(subj, eq_dist_LR, template_cifti, output_dir)
  


pixdim[1,2,3] should be non-zero; setting 0 dims to 1
pixdim[1,2,3] should be non-zero; setting 0 dims to 1
pixdim[1,2,3] should be non-zero; setting 0 dims to 1
pixdim[1,2,3] should be non-zero; setting 0 dims to 1
pixdim[1,2,3] should be non-zero; setting 0 dims to 1
pixdim[1,2,3] should be non-zero; setting 0 dims to 1
pixdim[1,2,3] should be non-zero; setting 0 dims to 1
pixdim[1,2,3] should be non-zero; setting 0 dims to 1
pixdim[1,2,3] should be non-zero; setting 0 dims to 1
pixdim[1,2,3] should be non-zero; setting 0 dims to 1
pixdim[1,2,3] should be non-zero; setting 0 dims to 1
pixdim[1,2,3] should be non-zero; setting 0 dims to 1
pixdim[1,2,3] should be non-zero; setting 0 dims to 1
pixdim[1,2,3] should be non-zero; setting 0 dims to 1
pixdim[1,2,3] should be non-zero; setting 0 dims to 1
pixdim[1,2,3] should be non-zero; setting 0 dims to 1
pixdim[1,2,3] should be non-zero; setting 0 dims to 1
pixdim[1,2,3] should be non-zero; setting 0 dims to 1
pixdim[1,2,3] should be non-

In [8]:
for i,subj in enumerate(subj_id):
    grad = nib.load(f'/home/fralberti/Data/Gradientile/{subj}.REST1_gcca_smooth6.dscalar.nii').get_fdata()[0]
    eq = nib.load(f'/home/fralberti/Data/Gradientile/{subj}_eqd.dscalar.nii').get_fdata()[0]
    r,p = stats.pearsonr(grad,eq)
    print(f'{subj}: ',r,p)

100206:  -0.42370405145318696 0.0
100307:  -0.4267517899230821 0.0
100408:  -0.43489163445000184 0.0
100610:  -0.4809475856047503 0.0
101006:  -0.3361242444870378 0.0
101107:  -0.39041996381121064 0.0
101309:  -0.40769518557595946 0.0
101410:  -0.39289364226475393 0.0
101915:  -0.3175605229632649 0.0
102008:  -0.38884225236752895 0.0
102109:  -0.40084581912309936 0.0
102311:  -0.42703355856801817 0.0
102513:  -0.37323399550033465 0.0
102614:  -0.361830469419108 0.0
102715:  -0.3052303367522637 0.0
